## **Mubeen Khan**
## **i170071@nu.edu.pk**

### **Installing & Importing Liberaries**

In [6]:
!pip install urduhack

     |████████████████████████████████| 112kB 6.8MB/s 
     |████████████████████████████████| 3.4MB 9.2MB/s 
     |████████████████████████████████| 686kB 40.1MB/s 
  Found existing installation: tensorflow-datasets 4.0.1
    Uninstalling tensorflow-datasets-4.0.1:
      Successfully uninstalled tensorflow-datasets-4.0.1


In [177]:
# importing necessary liberaries
import csv
from pandas import DataFrame, read_csv
import os
import urduhack
import math
from urduhack.preprocessing import remove_punctuation
from urduhack.preprocessing import remove_accents
from urduhack.preprocessing import normalize_whitespace
from urduhack.preprocessing import replace_numbers
from urduhack.preprocessing import replace_phone_numbers
from collections import Counter

### **Mounting Google Drive**

In [66]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### **Reading Files**

In [207]:
# function to read all files in a particular folder
def readAllFiles(folderName):
    allFiles = os.listdir(folderName)
    text = []

    for textFile in allFiles:
        if textFile.__contains__(".txt"):
            string = ''
            fileObject = open(folderName + textFile, "rt", encoding="utf-8")
            while(1):
                char = fileObject.read(1)
                if not char:
                    break
                elif char == '\n':
                    char = ' '
                string += char
            text.append(string)
    return text

In [208]:
# function to read file as tokens list
def readFile(fileName):
    with open(fileName, 'rt', encoding="utf-8") as f:
        reader = csv.reader(f)
        words = list(reader)
    f.close()
    return words

In [209]:
# reading train data files + stopwords
fakeText = readAllFiles('/content/drive/My Drive/NLP/A5/Train/Fake/')
print(len(fakeText))

realText = readAllFiles('/content/drive/My Drive/NLP/A5/Train/Real/')
print(len(realText))

stopWords = readFile('/content/drive/My Drive/NLP/A5/stopwords-ur.txt')
print(stopWords)
print(len(stopWords))

288
350
[['آئی'], ['آئے'], ['آج'], ['آخر'], ['آخرکبر'], ['آدهی'], ['آًب'], ['آٹھ'], ['آیب'], ['اة'], ['اخبزت'], ['اختتبم'], ['ادھر'], ['ارد'], ['اردگرد'], ['ارکبى'], ['اش'], ['اضتعوبل'], ['اضتعوبلات'], ['اضطرذ'], ['اضکب'], ['اضکی'], ['اضکے'], ['اطراف'], ['اغیب'], ['افراد'], ['الگ'], ['اور'], ['اوًچب'], ['اوًچبئی'], ['اوًچی'], ['اوًچے'], ['اى'], ['اً'], ['اًذر'], ['اًہیں'], ['اٹھبًب'], ['اپٌب'], ['اپٌے'], ['اچھب'], ['اچھی'], ['اچھے'], ['اکثر'], ['اکٹھب'], ['اکٹھی'], ['اکٹھے'], ['اکیلا'], ['اکیلی'], ['اکیلے'], ['اگرچہ'], ['اہن'], ['ایطے'], ['ایک'], ['ب'], ['ت'], ['تبزٍ'], ['تت'], ['تر'], ['ترتیت'], ['تریي'], ['تعذاد'], ['تن'], ['تو'], ['توبم'], ['توہی'], ['توہیں'], ['تٌہب'], ['تک'], ['تھب'], ['تھوڑا'], ['تھوڑی'], ['تھوڑے'], ['تھی'], ['تھے'], ['تیي'], ['ثب'], ['ثبئیں'], ['ثبترتیت'], ['ثبری'], ['ثبرے'], ['ثبعث'], ['ثبلا'], ['ثبلترتیت'], ['ثبہر'], ['ثدبئے'], ['ثرآں'], ['ثراں'], ['ثرش'], ['ثعذ'], ['ثغیر'], ['ثلٌذ'], ['ثلٌذوثبلا'], ['ثلکہ'], ['ثي'], ['ثٌب'], ['ثٌبرہب'], ['ثٌبرہی'], ['ثٌبرہے']

In [210]:
# reading test data files
testFakeText = readAllFiles('/content/drive/My Drive/NLP/A5/Test/Fake/')
print(len(fakeText))

testRealText = readAllFiles('/content/drive/My Drive/NLP/A5/Test/Real/')
print(len(realText))

288
350


### **Removing Duplicates for Boolean Na ̈ıve Bayes**

In [211]:
# function to remove duplication for each text given to it and returns a string having no duplicate words
def removeDuplicates(text):
    text = text.split(" ")  # tokenizing text 
  
    # joining two adjacent elements in iterable way
    for i in range(0, len(text)):
        text[i] = "".join(text[i])
          
    # creating dict. using counter method
    uniqueWords = Counter(text)
  
    # joining two adjacent elements in iterable way
    s = " ".join(uniqueWords.keys())
    return s

print(removeDuplicates("hello hi hello hello yo yo hi"), end='')

hello hi yo

In [212]:
# removing duplicates from train data and store the results in lists
noDupFText = []
noDupRText = []

for text in fakeText:
    noDupFText.append(removeDuplicates(text))

for text in realText:
    noDupRText.append(removeDuplicates(text))

### **Removing Stop Words**

In [213]:
# function to remove stop-words from the give list of text
def removeStopWords(text, stopWords):
    listWSW = []
    for t in text:
        s = ''
        tokens = t.split()
        for token in tokens:
            if (token not in stopWords):
                s += token + ' '
        if len(s) > 0:
            listWSW.append(s)
    return listWSW

x = ['hi', 'how', 'are', 'you', 'hello', 'world', 'hi', 'bro']
y = ['hi', 'bro']
z = removeStopWords(x, y)
print(z)

['how ', 'are ', 'you ', 'hello ', 'world ']


In [214]:
# removing stop-words from train data with and without duplicates and storing the results

fTextWSW = removeStopWords(fakeText, stopWords)
rTextWSW = removeStopWords(realText, stopWords)

noDupFTextWSW = removeStopWords(noDupFText, stopWords)
noDupRTextWSW = removeStopWords(noDupRText, stopWords)

### **Data Preprocessing (Data Cleaning)**

In [215]:
# fuction to remove unnecessary words from text
def dataPreprocessing(text):
    for i in range(len(text)):
        s = ''
        for c in text[i]:
            if (c.isdigit()):
                s += ' '
            else:
                s += c
        text[i] = s
        
    for i in range(len(text)):
        text[i] = normalize_whitespace(remove_punctuation(replace_numbers(remove_accents(text[i]))))

text = " سے   #@!#  230درجات پر مشمل20  فیصد"
l = [text]
dataPreprocessing(l)
print(l)

['سے درجات پر مشمل فیصد']


In [216]:
# funtion calling

dataPreprocessing(fakeText)
dataPreprocessing(realText)

dataPreprocessing(fTextWSW)
dataPreprocessing(rTextWSW)

dataPreprocessing(noDupFText)
dataPreprocessing(noDupRText)

dataPreprocessing(noDupFTextWSW)
dataPreprocessing(noDupRTextWSW)

dataPreprocessing(testFakeText)
dataPreprocessing(testRealText)

In [217]:
print (count(fakeText), count(fTextWSW))
print (count(realText), count(rTextWSW))
print (count(noDupFText), count(noDupFTextWSW))
print (count(noDupRText), count(noDupRTextWSW))

387913 387913
576608 576608
219856 219856
310372 310372


### **Tokenizing Text**

In [218]:
# function to tokenize given text
def tokenizeText(text):
    tokens = []
    for i in range(len(text)):
        tokens.extend(text[i].split())
    return tokens

x = ["hi how are you", " hello world", "hi bro 1234"]
y = tokenizeText(x)
print(y)

['hi', 'how', 'are', 'you', 'hello', 'world', 'hi', 'bro', '1234']


### **UniGram Dictionary Model**

In [219]:
# function to make unigram dict. with freq.
def unigramModel(tokens):
    dic = {}
    for token in tokens:
        if token in dic:
            dic[token] += 1
        else:
            dic[token] = 1
    return dic

x = ['hi', 'how', 'are', 'you', 'hello', 'world', 'hi', 'bro']
z = unigramModel(x)
print(z)

{'hi': 2, 'how': 1, 'are': 1, 'you': 1, 'hello': 1, 'world': 1, 'bro': 1}


### **Extracting Vocabolary**

In [220]:
# function to extract vocab. - I did it using x = unigramModel() because I calculated freq. which means x have unique keys, thus, less iterations were used
def extractVocabolary(text):
    vocab = []
    for i in range(len(text)):
        for token in text[i]:
            if token not in vocab:
                vocab.append(token)
    return vocab

x = ['hi','hi','hi','hi', 'how', 'are', 'you', 'hello', 'world', 'hi', 'bro']
v = extractVocabolary([x])
print(v)

['hi', 'how', 'are', 'you', 'hello', 'world', 'bro']


### **Na ̈ıve Bayes Classifier.**

In [221]:
# NaiveBayesTraining same as given in assignment.pdf
# this model takes 2 arguments: C = no. of classes, T = list of texts
# for this assignment, C = [0, 1] and T = [fakeText, RealText]
# Since the model is generic so I had only to modify T for all for classifiers
# classifiers = (Naive Bayes with Stop-Words, Naive Bayes without Stop-Words, Boolean Naive Bayes with Stop-Words, Boolean Naive Bayes without Stop-Words)

def NaiveBayesTraining(C, T):
    N = len(T[0]) + len(T[1])   # total text - (total files) - (fake, real)

    fTokens = tokenizeText(T[0])    # tokenizing fake text
    rTokens = tokenizeText(T[1])    # tokenizing real text
    tokensList = [fTokens, rTokens] # list of both

    V = extractVocabolary(tokensList) # total vocabolary

    fDict = unigramModel(fTokens)   # fake unigram model
    rDict = unigramModel(rTokens)   # real unigram model
    allDicts = [fDict, rDict]   # list of both

    prior = {}  # dict. to store priors
    condproc = [{},{}]  # list of dicts to store condproc for each class c

    for c in C:
        Nc = len(T[c])  # length of particular class words
        Nw = len(tokensList[c]) # length of all words
        prior[c] = Nc/N # calculating prior
        docc = allDicts[c]  # dict. of particular class - for freq. purpose
        for wi in docc:
            condproc[c][wi] = (allDicts[c][wi]+1)/(Nw + len(V)) # calculating conditional probability
    return V, prior, condproc

In [222]:
C = [0, 1]

T1 = [fakeText, realText]
T2 = [fTextWSW, rTextWSW]

T3 = [noDupFText, noDupRText]
T4 = [noDupFTextWSW, noDupRTextWSW]

NB = NaiveBayesTraining(C, T1)  # Naive Bayes with Stop-Words
NB_WSW = NaiveBayesTraining(C, T2)  # Naive Bayes without Stop-Words

BNB = NaiveBayesTraining(C, T3) # Boolean Naive Bayes with Stop-Words
BNB_WSW = NaiveBayesTraining(C, T4) # Boolean Naive Bayes without Stop-Words

### **Performance Evaluation Matrices**

In [223]:
# following function will predict whether a text is real or fake. Parameters are the text and the classifier type
def predictClassForText(text, classifier):
    words = text.split()
    checkList = []

    fakeProb = math.log2(NB[1][0])  # adding prior fake probability by using log to avoid underflowing
    trueProb = math.log2(NB[1][1])  # adding prior real probability by using log to avoid underflowing

    if classifier == "Naive Bayes with Stop-Words":
        for word in words:
            if word in NB[2][0]:
                fakeProb = math.log2(NB[2][0][word])
            if word in NB[2][1]:
                trueProb = math.log2(NB[2][1][word])

    elif classifier == "Naive Bayes without Stop-Words":
        for word in words:
            if word in NB_WSW[2][0]:
                fakeProb = math.log2(NB_WSW[2][0][word])
            if word in NB_WSW[2][1]:
                trueProb = math.log2(NB_WSW[2][1][word])

    elif classifier == "Boolean Naive Bayes with Stop-Words":
        for word in words:
            if word not in checkList:
                if word in BNB[2][0]:
                    fakeProb = math.log2(BNB[2][0][word])
                if word in BNB[2][1]:
                    trueProb = math.log2(BNB[2][1][word])
                checkList.append(word)

    elif classifier == "Boolean Naive Bayes without Stop-Words":
        for word in words:
            if word not in checkList:
                if word in BNB_WSW[2][0]:
                    fakeProb = math.log2(
                        noDupNB_WSW[2][0][word])
                if word in BNB_WSW[2][1]:
                    trueProb = math.log2(
                        noDupNB_WSW[2][1][word])
                checkList.append(word)

    # returning 0 for fake and 1 for real
    if fakeProb < trueProb:
        return 1
    return 0


In [224]:
# list to contain different classifier names
classifiers = ['Naive Bayes with Stop-Words', 'Naive Bayes without Stop-Words', 'Boolean Naive Bayes with Stop-Words', 'Boolean Naive Bayes without Stop-Words']

# predicting - returns 0 for fake and 1 for real
for classifier in classifiers:
    trueList = [] # for real test text - contains 0 if predicted fake, 1 for real
    wrongList = [] # for real test text - contains 0 if predicted fake, 1 for real

    for text in testFakeText:
        wrongList.append(predictClassForText(text, classifier))
    for text in testRealText:
        trueList.append(predictClassForText(text, classifier))

    # displaying matrix using formulas
    print("-----------------------------------------------------------\n")
    print("Classifier:\t", classifier)
    accuracy = (trueList.count(1) + wrongList.count(0)) / (len(trueList) + len(wrongList))
    print("\nAccuracy:\t", accuracy)
    recall = trueList.count(1) / len(trueList)
    print("Recall:\t\t", recall)
    precision = trueList.count(1) / (trueList.count(1) + wrongList.count(1))
    print("Precision:\t",precision)
    f_1 = (recall * precision * 2) / (precision + recall)
    print("F1 Measure:\t", f_1)
    print("-----------------------------------------------------------\n")

-----------------------------------------------------------

Classifier:	 Naive Bayes with Stop-Words

Accuracy:	 0.5190839694656488
Recall:		 0.5133333333333333
Precision:	 0.5923076923076923
F1 Measure:	 0.5499999999999999
-----------------------------------------------------------

-----------------------------------------------------------

Classifier:	 Naive Bayes without Stop-Words

Accuracy:	 0.5190839694656488
Recall:		 0.5133333333333333
Precision:	 0.5923076923076923
F1 Measure:	 0.5499999999999999
-----------------------------------------------------------

-----------------------------------------------------------

Classifier:	 Boolean Naive Bayes with Stop-Words

Accuracy:	 0.5534351145038168
Recall:		 0.6333333333333333
Precision:	 0.6050955414012739
F1 Measure:	 0.6188925081433224
-----------------------------------------------------------

-----------------------------------------------------------

Classifier:	 Boolean Naive Bayes without Stop-Words

Accuracy:	 0.5572

### **Report**




> **Procedure:**

*   Read fake and real train data files in 2 seperate lists (fake, real) where each list item represents each file text. Similarly, all files were read.
*   Read stop-words.txt in the form of list of tokens
*   Removed duplicates words from each text and stored results in 2 new lists.
*   Removed stop-words from each text and stored results in 2 new lists.
*   Preprossed data and remove unnecassary words.
*   Tokenized training data lists (fake, real).
*   Made unigram model for each training data list (fake, real).
*   Extracted vocabolary from all training data text.
*   Developed generic Naive Bayes Model
*   Evaluated performace


> **Accuracy:**

*   Naive Bayes With Stopwords : 51.9 %
*   Naive Bayes Without Stopwords : 51.9 %
*   Boolean Naive Bayes With Stopwords : 55.34 %
*   Boolean Naive Bayes Without Stopwords : 55.72 %